In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import io
import re
import tqdm
import string

In [ ]:
EMBEDDING_SIZE = 256
WINDOW_SIZE = 1

In [ ]:
# Define the corpus
def custom_standardization(input_data):
    text = input_data.lower()

    # 2. remove punctuation (keep letters, digits, spaces)
    text = re.sub(r"[^\w\s]", "", text, flags=re.UNICODE)

    # 3. replace newlines with spaces and collapse extra spaces
    text = re.sub(r"\s+", " ", text)
    return text


f = open("war_and_peace.txt", "r")
input_data = f.read()
data = custom_standardization(input_data)
corpus = [data]
 
# Convert the corpus to a sequence of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
# print("After converting our words in the corpus into vector of integers:")
# print(len(sequences[0]))

word_index = tokenizer.word_index
index_word = {v: k for k, v in word_index.items()}

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

words = data.split()
contexts = []
targets = []
for i in range(1, len(words)):
    x_key = word_index.get(words[i-1], None)
    y_key = word_index.get(words[i], None)
    
    if x_key is not None and y_key is not None:
        contexts.append(x_key)
        targets.append(y_key)
    else:
        continue
        
vocab_size = len(tokenizer.word_index) + 1

# Convert the contexts and targets to numpy arrays
X = np.array(contexts)

y = np.zeros((len(X), vocab_size))
for i in range(0, len(targets)):
    y[i][targets[i]] = 1

In [ ]:
# Define the CBOW model
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=EMBEDDING_SIZE,
                    input_length=1))
model.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
# Fit model
# model.fit(X[:1000], y, epochs=5, verbose=2)